In [0]:
pip install python-dotenv

In [0]:
# Load additional environment variables
# (OAuth authentication is already set up in the previous cell)
from dotenv import load_dotenv
import os
    
# We already loaded .env file in the authentication cell, but we'll do it again to be safe
load_dotenv()

# Keep these variables for other operations as needed
DATABRICKS_INSTANCE = os.getenv("DATABRICKS_INSTANCE")
CLUSTER_ID = os.getenv("CLUSTER_ID")
WAREHOUSE_ID = os.getenv("WAREHOUSE_ID")

# ACCOUNT_ID is already loaded in the authentication cell
# We'll keep it here for reference and backward compatibility
ACCOUNT_ID = os.getenv("ACCOUNT_ID")

# Get credentials from environment variables
TOKEN = os.getenv("TOKEN")
DATABRICKS_INSTANCE = os.getenv("DATABRICKS_INSTANCE")

# Display environment variables for reference
print(f"DATABRICKS_INSTANCE: {DATABRICKS_INSTANCE}")
print(f"CLUSTER_ID: {CLUSTER_ID}")
print(f"WAREHOUSE_ID: {WAREHOUSE_ID}")
print(f"ACCOUNT_ID: {ACCOUNT_ID}")

In [0]:
# Authentication using Personal Access Token (PAT) for Databricks Workspace APIs
import requests
import json
import os


def test_authentication():
    """
    Test authentication by making a simple API call to list cluster policies.
    """
    if not TOKEN:
        print("ERROR: TOKEN environment variable is not set.")
        return False
        
    if not DATABRICKS_INSTANCE:
        print("ERROR: DATABRICKS_INSTANCE environment variable is not set.")
        return False
    
    # Use the token to make a simple API call to list cluster policies
    url = f"https://{DATABRICKS_INSTANCE}/api/2.0/policies/clusters/list"
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            print("✅ Authentication successful!")
            # Print a sample of the response (first few policies if any)
            policies = response.json().get("policies", [])
            if policies:
                print(f"Found {len(policies)} cluster policies.")
                if len(policies) > 0:
                    print(f"First policy name: {policies[0].get('name', 'N/A')}")
            else:
                print("No cluster policies found, but API call was successful.")
            return True
        else:
            print(f"❌ Authentication failed with status code: {response.status_code}")
            print(response.text)
            return False
    except Exception as e:
        print(f"❌ Error testing authentication: {str(e)}")
        return False

# Test authentication
auth_success = test_authentication()
print(f"Authentication status: {'Successful' if auth_success else 'Failed'}")

# Define the make_api_request function to use the PAT token
def make_api_request(method, endpoint, data=None, params=None):
    """
    Make a request to the Databricks Workspace API using PAT authentication.
    
    Args:
        method (str): HTTP method (GET, POST, PATCH, DELETE, PUT)
        endpoint (str): API endpoint path
        data (dict, optional): Request payload
        params (dict, optional): Query parameters
        
    Returns:
        dict: Response from the API
    """
    url = f"https://{DATABRICKS_INSTANCE}{endpoint}"
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.request(
            method=method,
            url=url,
            headers=headers,
            data=json.dumps(data) if data else None,
            params=params
        )
        
        if response.status_code in [200, 201, 202, 204]:
            return response.json() if response.content else {}
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return None
    except Exception as e:
        print(f"Error making API request: {str(e)}")
        return None

### List Cluster Policies

In [0]:
def list_cluster_policies(sort_column=None, sort_order=None):
    """
    List all cluster policies accessible by the requesting user.
    
    Args:
        sort_column (str, optional): The cluster policy attribute to sort by.
            Valid values: "POLICY_CREATION_TIME", "POLICY_NAME"
        sort_order (str, optional): The order in which the policies get listed.
            Valid values: "DESC", "ASC". Default is "DESC".
            
    Returns:
        dict: List of cluster policies
    """
    endpoint = "/api/2.0/policies/clusters/list"
    params = {}
    
    if sort_column:
        params["sort_column"] = sort_column
    if sort_order:
        params["sort_order"] = sort_order
    
    return make_api_request("GET", endpoint, params=params)

# Example: List all cluster policies
policies = list_cluster_policies()
print(json.dumps(policies, indent=2))

### Create Cluster Policy

In [0]:
def create_cluster_policy(name, definition=None, description=None, max_clusters_per_user=None, 
                          policy_family_id=None, policy_family_definition_overrides=None, libraries=None):
    """
    Create a new cluster policy.
    
    Args:
        name (str): Policy name (1-100 chars), must be unique
        definition (str, optional): Policy definition JSON as a string
        description (str, optional): Human-readable description (max 1000 chars)
        max_clusters_per_user (int, optional): Max number of clusters per user
        policy_family_id (str, optional): ID of policy family to inherit from
        policy_family_definition_overrides (str, optional): Customizations to family policy
        libraries (list, optional): List of library configs to install
            
    Returns:
        dict: Created cluster policy ID
    
    Note: 
    - Cannot use both definition and policy_family_id together
    - If using policy_family_id, use policy_family_definition_overrides to customize
    """
    endpoint = "/api/2.0/policies/clusters/create"
    
    # Start with required fields
    data = {
        "name": name
    }
    
    # Add optional fields if provided
    if definition:
        data["definition"] = definition
    
    if description:
        data["description"] = description
    
    if max_clusters_per_user is not None:
        data["max_clusters_per_user"] = max_clusters_per_user
    
    if policy_family_id:
        data["policy_family_id"] = policy_family_id
    
    if policy_family_definition_overrides:
        data["policy_family_definition_overrides"] = policy_family_definition_overrides
    
    if libraries:
        data["libraries"] = libraries
    
    return make_api_request("POST", endpoint, data=data)

# Example: Create a new cluster policy
# Uncomment and modify as needed
policy_definition = """{
  "spark_version": {
    "type": "fixed",
    "value": "11.3.x-scala2.12"
  },
  "node_type_id": {
    "type": "allowlist",
    "values": ["Standard_DS3_v2", "Standard_DS4_v2"]
  },
  "autotermination_minutes": {
    "type": "fixed",
    "value": 120
  },
  "custom_tags.team": {
    "type": "fixed",
    "value": "data_engineering"
  }
}"""

new_policy = create_cluster_policy(
    name="DE Team Standard Cluster Policy",
    definition=policy_definition, 
    description="Standard policy for Data Engineering team",
    max_clusters_per_user=5
)
print(json.dumps(new_policy, indent=2))

### Get Cluster Policy by ID

In [0]:
def get_cluster_policy(policy_id):
    """
    Get a cluster policy by ID.
    
    Args:
        policy_id (str): ID of the policy to retrieve
        
    Returns:
        dict: Cluster policy details
    """
    endpoint = "/api/2.0/policies/clusters/get"
    params = {"policy_id": policy_id}
    
    return make_api_request("GET", endpoint, params=params)

# Example: Get a cluster policy by ID
# Uncomment and modify as needed
policy_id = "000697660EBE35F0"  # Replace with actual policy ID
policy = get_cluster_policy(policy_id)
print(json.dumps(policy, indent=2))

### Update Cluster Policy

In [0]:
def update_cluster_policy(policy_id, name=None, definition=None, description=None, 
                          max_clusters_per_user=None, policy_family_id=None, 
                          policy_family_definition_overrides=None, libraries=None):
    """
    Update an existing cluster policy.
    
    Args:
        policy_id (str): ID of the policy to update
        name (str, optional): New name for the policy
        definition (str, optional): New policy definition JSON as a string
        description (str, optional): New human-readable description
        max_clusters_per_user (int, optional): New max clusters per user
        policy_family_id (str, optional): New policy family ID
        policy_family_definition_overrides (str, optional): New family policy customizations
        libraries (list, optional): New list of library configs
            
    Returns:
        dict: Empty dict if successful
    
    Note: 
    - Cannot use both definition and policy_family_id together
    - If using policy_family_id, use policy_family_definition_overrides to customize
    """
    endpoint = "/api/2.0/policies/clusters/edit"
    
    # Start with required field
    data = {
        "policy_id": policy_id
    }
    
    # Add optional fields if provided
    if name:
        data["name"] = name
    
    if definition:
        data["definition"] = definition
    
    if description:
        data["description"] = description
    
    if max_clusters_per_user is not None:
        data["max_clusters_per_user"] = max_clusters_per_user
    
    if policy_family_id:
        data["policy_family_id"] = policy_family_id
    
    if policy_family_definition_overrides:
        data["policy_family_definition_overrides"] = policy_family_definition_overrides
    
    if libraries:
        data["libraries"] = libraries
    
    return make_api_request("POST", endpoint, data=data)

# Example: Update a cluster policy
# Uncomment and modify as needed
policy_id = "000697660EBE35F0"  # Replace with actual policy ID
updated_policy_definition = """{
  "spark_version": {
    "type": "fixed",
    "value": "11.3.x-scala2.12"
  },
  "node_type_id": {
    "type": "allowlist",
    "values": ["Standard_DS3_v2", "Standard_DS4_v2", "Standard_DS5_v2"]
  },
  "autotermination_minutes": {
    "type": "fixed",
    "value": 120
  },
  "custom_tags.team": {
    "type": "fixed",
    "value": "data_engineering"
  },
  "custom_tags.environment": {
    "type": "fixed",
    "value": "development"
  }
}"""

updated_policy = update_cluster_policy(
    policy_id=policy_id,
    name="Updated DE Team Cluster Policy",
    definition=updated_policy_definition,
    max_clusters_per_user=10
)
print(json.dumps(updated_policy, indent=2))

### Delete Cluster Policy

In [0]:
def delete_cluster_policy(policy_id):
    """
    Delete a cluster policy.
    
    Args:
        policy_id (str): ID of the policy to delete
        
    Returns:
        dict: Empty dict if successful
    """
    endpoint = "/api/2.0/policies/clusters/delete"
    data = {"policy_id": policy_id}
    
    return make_api_request("POST", endpoint, data=data)

# Example: Delete a cluster policy
# Uncomment and modify as needed
policy_id = "000697660EBE35F0"  # Replace with actual policy ID
result = delete_cluster_policy(policy_id)
if result is not None:
    print(f"Policy {policy_id} deleted successfully.")
else:
    print(f"Failed to delete policy {policy_id}.")

### Get Cluster Policy Permissions

In [0]:
def get_cluster_policy_permissions(cluster_policy_id):
    """
    Get permissions for a cluster policy.
    
    Args:
        cluster_policy_id (str): The ID of the cluster policy
        
    Returns:
        dict: Permissions information including access control list
    """
    endpoint = f"/api/2.0/permissions/cluster-policies/{cluster_policy_id}"
    return make_api_request("GET", endpoint)

# Example: Get permissions for a cluster policy
# Uncomment and modify as needed
policy_id = "001D75C71DD3EBE1"  # Replace with actual policy ID
permissions = get_cluster_policy_permissions(policy_id)
print(json.dumps(permissions, indent=2))

### Set Cluster Policy Permissions

In [0]:
def set_cluster_policy_permissions(cluster_policy_id, access_control_list):
    """
    Set permissions on a cluster policy, replacing existing permissions.
    
    Args:
        cluster_policy_id (str): The ID of the cluster policy
        access_control_list (list): List of access control items with structure:
            [
                {
                    "user_name": "string",  # Optional, either user_name, group_name, or service_principal_name
                    "group_name": "string",  # Optional
                    "service_principal_name": "string",  # Optional
                    "permission_level": "CAN_USE"  # Required
                }
            ]
        
    Returns:
        dict: Updated permissions information
    """
    endpoint = f"/api/2.0/permissions/cluster-policies/{cluster_policy_id}"
    data = {"access_control_list": access_control_list}
    
    return make_api_request("PUT", endpoint, data=data)

# Example: Set permissions for a cluster policy
# Uncomment and modify as needed
policy_id = "001D75C71DD3EBE1"  # Replace with actual policy ID
new_permissions = [
    {
        "group_name": "data-engineers",
        "permission_level": "CAN_USE"
    },
    {
        "user_name": "admin@example.com",
        "permission_level": "CAN_MANAGE"
    }
]

result = set_cluster_policy_permissions(policy_id, new_permissions)
print(json.dumps(result, indent=2))

### Update Cluster Policy Permissions

In [0]:
def update_cluster_policy_permissions(cluster_policy_id, access_control_list):
    """
    Update permissions on a cluster policy, merging with existing permissions.
    
    Args:
        cluster_policy_id (str): The ID of the cluster policy
        access_control_list (list): List of access control items to update
        
    Returns:
        dict: Updated permissions information
    """
    endpoint = f"/api/2.0/permissions/cluster-policies/{cluster_policy_id}"
    data = {"access_control_list": access_control_list}
    
    return make_api_request("PATCH", endpoint, data=data)

# Example: Update permissions for a cluster policy
# Uncomment and modify as needed
policy_id = "001D75C71DD3EBE1"  # Replace with actual policy ID
permission_updates = [
    {
        "group_name": "data-scientists",
        "permission_level": "CAN_USE"
    }
]

result = update_cluster_policy_permissions(policy_id, permission_updates)
print(json.dumps(result, indent=2))

### Get Cluster Policy Permission Levels

In [0]:
def get_cluster_policy_permission_levels(cluster_policy_id):
    """
    Get the permission levels that can be granted on a cluster policy.
    
    Args:
        cluster_policy_id (str): The ID of the cluster policy
        
    Returns:
        dict: Available permission levels
    """
    endpoint = f"/api/2.0/permissions/cluster-policies/{cluster_policy_id}/permissionLevels"
    return make_api_request("GET", endpoint)

# Example: Get permission levels for a cluster policy
# Uncomment and modify as needed
policy_id = "001D75C71DD3EBE1"  # Replace with actual policy ID
permission_levels = get_cluster_policy_permission_levels(policy_id)
print(json.dumps(permission_levels, indent=2))

### Helper Functions

In [0]:
def prettify_policy(policy):
    """
    Format a cluster policy for better readability.
    
    Args:
        policy (dict): Cluster policy object
        
    Returns:
        str: Formatted policy string
    """
    output = []
    output.append(f"Policy ID: {policy.get('policy_id', 'N/A')}")
    output.append(f"Name: {policy.get('name', 'N/A')}")
    output.append(f"Created at: {policy.get('created_at_timestamp', 'N/A')}")
    output.append(f"Creator: {policy.get('creator_user_name', 'N/A')}")
    
    if policy.get('description'):
        output.append(f"Description: {policy.get('description')}")
    
    if policy.get('max_clusters_per_user'):
        output.append(f"Max clusters per user: {policy.get('max_clusters_per_user')}")
    
    if policy.get('policy_family_id'):
        output.append(f"Policy family ID: {policy.get('policy_family_id')}")
    
    if policy.get('definition'):
        # Try to pretty-print the definition if it's a string
        if isinstance(policy.get('definition'), str):
            try:
                definition_obj = json.loads(policy.get('definition'))
                output.append(f"Definition: {json.dumps(definition_obj, indent=2)}")
            except:
                output.append(f"Definition: {policy.get('definition')}")
        else:
            # If it's already a dict
            output.append(f"Definition: {json.dumps(policy.get('definition'), indent=2)}")
    
    if policy.get('libraries'):
        output.append("Libraries:")
        for i, lib in enumerate(policy.get('libraries')):
            output.append(f"  {i+1}. {lib}")
    
    return "\n".join(output)

def display_policies(policies_response):
    """
    Display a list of policies in a more readable format.
    
    Args:
        policies_response (dict): Response from list_cluster_policies
    """
    if not policies_response:
        print("No response or error occurred.")
        return
    
    policies = policies_response.get('policies', [])
    
    print(f"Total policies: {len(policies)}")
    
    print("\nPolicies:")
    for i, policy in enumerate(policies, 1):
        print(f"\n{i}. " + prettify_policy(policy))
        print("-" * 50)

def format_permissions(permissions):
    """
    Format permissions for better readability.
    
    Args:
        permissions (dict): Permissions object
        
    Returns:
        str: Formatted permissions string
    """
    if not permissions:
        return "No permissions data available."
    
    output = []
    output.append(f"Object ID: {permissions.get('object_id', 'N/A')}")
    output.append(f"Object Type: {permissions.get('object_type', 'N/A')}")
    
    acl = permissions.get('access_control_list', [])
    if acl:
        output.append("\nAccess Control List:")
        
        for i, entry in enumerate(acl, 1):
            output.append(f"\n  {i}. Principal: ", end="")
            
            if 'user_name' in entry:
                output.append(f"User '{entry['user_name']}'")
            elif 'group_name' in entry:
                output.append(f"Group '{entry['group_name']}'")
            elif 'service_principal_name' in entry:
                output.append(f"Service Principal '{entry['service_principal_name']}'")
            
            if 'display_name' in entry:
                output.append(f" (Display name: {entry['display_name']})")
            
            if 'all_permissions' in entry:
                for perm in entry['all_permissions']:
                    inherited = "Inherited" if perm.get('inherited', False) else "Direct"
                    source = f" from {', '.join(perm.get('inherited_from_object', []))}" if perm.get('inherited', False) else ""
                    output.append(f"    - {perm.get('permission_level', 'Unknown')} ({inherited}{source})")
    
    return "\n".join(output)

# Example: Use the helper functions
all_policies = list_cluster_policies()
display_policies(all_policies)

### Validation Functions

In [0]:
def validate_policy_name(name):
    """
    Validate policy name according to API requirements.
    
    Args:
        name (str): Policy name to validate
        
    Returns:
        bool: True if valid, False otherwise
    """
    if not name:
        print("Error: Policy name cannot be empty.")
        return False
    
    if len(name) > 100:
        print("Error: Policy name must be at most 100 characters.")
        return False
    
    return True

def validate_policy_definition(definition):
    """
    Validate policy definition JSON.
    
    Args:
        definition (str): Policy definition JSON string
        
    Returns:
        bool: True if valid, False otherwise
    """
    if not definition:
        print("Error: Policy definition cannot be empty.")
        return False
    
    try:
        # Check if the definition is valid JSON
        json.loads(definition)
        return True
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in policy definition: {e}")
        return False

def validate_policy_description(description):
    """
    Validate policy description.
    
    Args:
        description (str): Policy description
        
    Returns:
        bool: True if valid, False otherwise
    """
    if description and len(description) > 1000:
        print("Error: Policy description must be at most 1000 characters.")
        return False
    
    return True

def validate_libraries(libraries):
    """
    Validate libraries list.
    
    Args:
        libraries (list): List of libraries to validate
        
    Returns:
        bool: True if valid, False otherwise
    """
    if not libraries:
        return True
    
    if not isinstance(libraries, list):
        print("Error: Libraries must be a list.")
        return False
    
    if len(libraries) > 500:
        print("Error: At most 500 libraries are allowed per policy.")
        return False
    
    for lib in libraries:
        if not isinstance(lib, dict):
            print("Error: Each library must be a dictionary.")
            return False
        
        valid_lib_types = ['jar', 'egg', 'whl', 'pypi', 'maven', 'cran', 'requirements']
        if not any(lib_type in lib for lib_type in valid_lib_types):
            print(f"Error: Each library must have at least one of these keys: {', '.join(valid_lib_types)}")
            return False
    
    return True

def validate_max_clusters_per_user(max_clusters):
    """
    Validate max_clusters_per_user.
    
    Args:
        max_clusters (int): Max clusters per user
        
    Returns:
        bool: True if valid, False otherwise
    """
    if max_clusters is not None:
        if not isinstance(max_clusters, int):
            print("Error: max_clusters_per_user must be an integer.")
            return False
        
        if max_clusters < 1:
            print("Error: max_clusters_per_user must be at least 1.")
            return False
    
    return True

def validate_permission_level(permission_level):
    """
    Validate permission level for cluster policies.
    
    Args:
        permission_level (str): Permission level
        
    Returns:
        bool: True if valid, False otherwise
    """
    valid_levels = ["CAN_USE", "CAN_MANAGE"]
    
    if permission_level not in valid_levels:
        print(f"Error: permission_level must be one of: {', '.join(valid_levels)}")
        return False
    
    return True

### End-to-End Example

In [0]:
def run_complete_example():
    """
    Complete example of cluster policy management workflow.
    """
    # Step 1: Create a new policy
    print("Creating a new cluster policy...")
    
    policy_definition = """{
      "spark_version": {
        "type": "fixed",
        "value": "11.3.x-scala2.12"
      },
      "node_type_id": {
        "type": "fixed",
        "value": "Standard_DS3_v2"
      },
      "num_workers": {
        "type": "range",
        "min_value": 1,
        "max_value": 5
      },
      "autotermination_minutes": {
        "type": "fixed",
        "value": 60
      },
      "custom_tags.environment": {
        "type": "fixed",
        "value": "dev"
      },
      "custom_tags.team": {
        "type": "fixed",
        "value": "data_science"
      }
    }"""
    
    # Validate inputs
    if not validate_policy_name("Example Data Science Policy"):
        return
    if not validate_policy_definition(policy_definition):
        return
    
    new_policy = create_cluster_policy(
        name="Example Data Science Policy",
        definition=policy_definition,
        description="Example policy for data scientists",
        max_clusters_per_user=3
    )
    
    if not new_policy:
        print("Failed to create a policy. Exiting example.")
        return
    
    policy_id = new_policy.get("policy_id")
    print(f"Created policy with ID: {policy_id}")
    
    # Step 2: Get the policy details
    print("\nFetching the policy details...")
    policy = get_cluster_policy(policy_id)
    if policy:
        print(prettify_policy(policy))
    
    # Step 3: Update the policy
    print("\nUpdating the policy...")
    
    updated_definition = """{
      "spark_version": {
        "type": "fixed",
        "value": "11.3.x-scala2.12"
      },
      "node_type_id": {
        "type": "fixed",
        "value": "Standard_DS3_v2"
      },
      "num_workers": {
        "type": "range",
        "min_value": 2,
        "max_value": 8
      },
      "autotermination_minutes": {
        "type": "fixed",
        "value": 90
      },
      "custom_tags.environment": {
        "type": "fixed",
        "value": "dev"
      },
      "custom_tags.team": {
        "type": "fixed",
        "value": "data_science"
      },
      "custom_tags.project": {
        "type": "fixed",
        "value": "customer_analytics"
      }
    }"""
    
    # Validate inputs
    if not validate_policy_name("Updated Data Science Policy"):
        return
    if not validate_policy_definition(updated_definition):
        return
    
    updated_policy = update_cluster_policy(
        policy_id=policy_id,
        name="Updated Data Science Policy",
        definition=updated_definition,
        description="Updated example policy for data scientists",
        max_clusters_per_user=5
    )
    
    if updated_policy is not None:
        print("Policy updated successfully.")
        policy = get_cluster_policy(policy_id)
        if policy:
            print(prettify_policy(policy))
    
    # Step 4: Set permissions on the policy
    print("\nSetting permissions on the policy...")
    
    permissions = [
        {
            "group_name": "data-scientists",
            "permission_level": "CAN_USE"
        },
        {
            "user_name": "admin@example.com",
            "permission_level": "CAN_MANAGE"
        }
    ]
    
    perm_result = set_cluster_policy_permissions(policy_id, permissions)
    if perm_result:
        print("Permissions set successfully.")
        print(format_permissions(perm_result))
    
    # Step 5: List all policies
    print("\nListing all policies...")
    all_policies = list_cluster_policies()
    display_policies(all_policies)
    
    # Step 6: Delete the policy
    print(f"\nDeleting policy {policy_id}...")
    confirm = input(f"Delete policy {policy_id}? (y/n): ")
    
    if confirm.lower() == 'y':
        result = delete_cluster_policy(policy_id)
        if result is not None:
            print(f"Policy {policy_id} deleted successfully.")
            
            # Verify deletion
            print("\nVerifying deletion by listing all policies again...")
            all_policies_after_deletion = list_cluster_policies()
            display_policies(all_policies_after_deletion)
        else:
            print(f"Failed to delete policy {policy_id}.")
    else:
        print("Policy deletion cancelled.")

# Uncomment to run the complete example
run_complete_example()

### CLI for Cluster Policy Management

In [0]:
def cluster_policy_cli():
    """
    Interactive command-line interface for managing cluster policies.
    """
    while True:
        print("\n=== Cluster Policy Management CLI ===")
        print("1. List all policies")
        print("2. Create a new policy")
        print("3. Get a policy by ID")
        print("4. Update a policy")
        print("5. Delete a policy")
        print("6. Get policy permissions")
        print("7. Set policy permissions")
        print("8. Update policy permissions")
        print("9. Exit")
        
        choice = input("\nEnter your choice (1-9): ")
        
        if choice == '1':
            # List policies with optional sorting
            sort_column = input("Enter sort column (POLICY_CREATION_TIME/POLICY_NAME, press Enter for default): ")
            sort_column = sort_column if sort_column in ["POLICY_CREATION_TIME", "POLICY_NAME"] else None
            
            sort_order = input("Enter sort order (DESC/ASC, press Enter for default): ")
            sort_order = sort_order if sort_order in ["DESC", "ASC"] else None
            
            policies = list_cluster_policies(sort_column=sort_column, sort_order=sort_order)
            display_policies(policies)
        
        elif choice == '2':
            # Create new policy
            name = input("Enter policy name: ")
            
            if not validate_policy_name(name):
                continue
            
            description = input("Enter policy description (press Enter for none): ")
            
            if description and not validate_policy_description(description):
                continue
            
            max_clusters = input("Enter max clusters per user (press Enter for none): ")
            if max_clusters:
                try:
                    max_clusters = int(max_clusters)
                    if not validate_max_clusters_per_user(max_clusters):
                        continue
                except ValueError:
                    print("Error: max_clusters_per_user must be an integer.")
                    continue
            else:
                max_clusters = None
            
            use_family = input("Use policy family? (y/n): ").lower() == 'y'
            
            if use_family:
                policy_family_id = input("Enter policy family ID: ")
                definition = None
                
                family_overrides = input("Enter policy family definition overrides (valid JSON, press Enter for none): ")
                if family_overrides and not validate_policy_definition(family_overrides):
                    continue
                policy_family_definition_overrides = family_overrides if family_overrides else None
            else:
                policy_family_id = None
                policy_family_definition_overrides = None
                
                definition = input("Enter policy definition (valid JSON): ")
                if not validate_policy_definition(definition):
                    continue
            
            add_libraries = input("Add libraries? (y/n): ").lower() == 'y'
            libraries = []
            
            if add_libraries:
                lib_count = input("Enter number of libraries: ")
                try:
                    lib_count = int(lib_count)
                    
                    for i in range(lib_count):
                        print(f"\nLibrary {i+1}:")
                        lib_type = input("Library type (jar/egg/whl/pypi/maven/cran/requirements): ")
                        
                        if lib_type == 'jar':
                            lib_path = input("JAR path: ")
                            libraries.append({"jar": lib_path})
                        elif lib_type == 'egg':
                            lib_path = input("EGG path: ")
                            libraries.append({"egg": lib_path})
                        elif lib_type == 'whl':
                            lib_path = input("WHL path: ")
                            libraries.append({"whl": lib_path})
                        elif lib_type == 'pypi':
                            package = input("PyPI package: ")
                            repo = input("PyPI repo (press Enter for default): ")
                            lib = {"pypi": {"package": package}}
                            if repo:
                                lib["pypi"]["repo"] = repo
                            libraries.append(lib)
                        elif lib_type == 'maven':
                            coordinates = input("Maven coordinates: ")
                            repo = input("Maven repo (press Enter for default): ")
                            lib = {"maven": {"coordinates": coordinates}}
                            if repo:
                                lib["maven"]["repo"] = repo
                            libraries.append(lib)
                        elif lib_type == 'cran':
                            package = input("CRAN package: ")
                            repo = input("CRAN repo (press Enter for default): ")
                            lib = {"cran": {"package": package}}
                            if repo:
                                lib["cran"]["repo"] = repo
                            libraries.append(lib)
                        elif lib_type == 'requirements':
                            req_path = input("Requirements path: ")
                            libraries.append({"requirements": req_path})
                        else:
                            print(f"Unsupported library type: {lib_type}")
                            continue
                except ValueError:
                    print("Error: Number of libraries must be an integer.")
                    continue
            
            if libraries and not validate_libraries(libraries):
                continue
            
            new_policy = create_cluster_policy(
                name=name,
                definition=definition,
                description=description if description else None,
                max_clusters_per_user=max_clusters,
                policy_family_id=policy_family_id,
                policy_family_definition_overrides=policy_family_definition_overrides,
                libraries=libraries if libraries else None
            )
            
            if new_policy:
                print("\nPolicy created successfully:")
                policy_id = new_policy.get("policy_id")
                policy = get_cluster_policy(policy_id)
                if policy:
                    print(prettify_policy(policy))
            else:
                print("\nFailed to create policy.")
        
        elif choice == '3':
            # Get policy by ID
            policy_id = input("Enter policy ID: ")
            policy = get_cluster_policy(policy_id)
            
            if policy:
                print("\nPolicy details:")
                print(prettify_policy(policy))
            else:
                print(f"\nPolicy with ID {policy_id} not found or error occurred.")
        
        elif choice == '4':
            # Update existing policy
            policy_id = input("Enter policy ID to update: ")
            
            # Get current policy to show current values
            current_policy = get_cluster_policy(policy_id)
            
            if not current_policy:
                print(f"\nPolicy with ID {policy_id} not found or error occurred.")
                continue
            
            print("\nCurrent policy details:")
            print(prettify_policy(current_policy))
            
            # Get updated values
            name = input(f"\nEnter new policy name (press Enter to keep '{current_policy.get('name')}'): ")
            name = name if name else current_policy.get('name')
            
            if not validate_policy_name(name):
                continue
            
            description = input(f"Enter new description (press Enter to keep current): ")
            description = description if description else current_policy.get('description')
            
            if description and not validate_policy_description(description):
                continue
            
            max_clusters_str = input(f"Enter new max clusters per user (press Enter to keep current): ")
            if max_clusters_str:
                try:
                    max_clusters = int(max_clusters_str)
                    if not validate_max_clusters_per_user(max_clusters):
                        continue
                except ValueError:
                    print("Error: max_clusters_per_user must be an integer.")
                    continue
            else:
                max_clusters = current_policy.get('max_clusters_per_user')
            
            update_definition = input("Update policy definition? (y/n): ").lower() == 'y'
            
            if update_definition:
                use_family = input("Use policy family? (y/n): ").lower() == 'y'
                
                if use_family:
                    policy_family_id = input("Enter policy family ID: ")
                    definition = None
                    
                    family_overrides = input("Enter policy family definition overrides (valid JSON, press Enter for none): ")
                    if family_overrides and not validate_policy_definition(family_overrides):
                        continue
                    policy_family_definition_overrides = family_overrides if family_overrides else None
                else:
                    policy_family_id = None
                    policy_family_definition_overrides = None
                    
                    definition = input("Enter new policy definition (valid JSON): ")
                    if not validate_policy_definition(definition):
                        continue
            else:
                definition = current_policy.get('definition')
                policy_family_id = current_policy.get('policy_family_id')
                policy_family_definition_overrides = current_policy.get('policy_family_definition_overrides')
            
            update_libraries = input("Update libraries? (y/n): ").lower() == 'y'
            
            if update_libraries:
                add_libraries = input("Add libraries? (y/n): ").lower() == 'y'
                libraries = []
                
                if add_libraries:
                    lib_count = input("Enter number of libraries: ")
                    try:
                        lib_count = int(lib_count)
                        
                        for i in range(lib_count):
                            print(f"\nLibrary {i+1}:")
                            lib_type = input("Library type (jar/egg/whl/pypi/maven/cran/requirements): ")
                            
                            if lib_type == 'jar':
                                lib_path = input("JAR path: ")
                                libraries.append({"jar": lib_path})
                            elif lib_type == 'egg':
                                lib_path = input("EGG path: ")
                                libraries.append({"egg": lib_path})
                            elif lib_type == 'whl':
                                lib_path = input("WHL path: ")
                                libraries.append({"whl": lib_path})
                            elif lib_type == 'pypi':
                                package = input("PyPI package: ")
                                repo = input("PyPI repo (press Enter for default): ")
                                lib = {"pypi": {"package": package}}
                                if repo:
                                    lib["pypi"]["repo"] = repo
                                libraries.append(lib)
                            elif lib_type == 'maven':
                                coordinates = input("Maven coordinates: ")
                                repo = input("Maven repo (press Enter for default): ")
                                lib = {"maven": {"coordinates": coordinates}}
                                if repo:
                                    lib["maven"]["repo"] = repo
                                libraries.append(lib)
                            elif lib_type == 'cran':
                                package = input("CRAN package: ")
                                repo = input("CRAN repo (press Enter for default): ")
                                lib = {"cran": {"package": package}}
                                if repo:
                                    lib["cran"]["repo"] = repo
                                libraries.append(lib)
                            elif lib_type == 'requirements':
                                req_path = input("Requirements path: ")
                                libraries.append({"requirements": req_path})
                            else:
                                print(f"Unsupported library type: {lib_type}")
                                continue
                    except ValueError:
                        print("Error: Number of libraries must be an integer.")
                        continue
                
                if libraries and not validate_libraries(libraries):
                    continue
            else:
                libraries = current_policy.get('libraries')
            
            updated_policy = update_cluster_policy(
                policy_id=policy_id,
                name=name,
                definition=definition,
                description=description,
                max_clusters_per_user=max_clusters,
                policy_family_id=policy_family_id,
                policy_family_definition_overrides=policy_family_definition_overrides,
                libraries=libraries
            )
            
            if updated_policy is not None:
                print("\nPolicy updated successfully.")
                policy = get_cluster_policy(policy_id)
                if policy:
                    print(prettify_policy(policy))
            else:
                print("\nFailed to update policy.")
        
        elif choice == '5':
            # Delete policy
            policy_id = input("Enter policy ID to delete: ")
            
            # Confirm deletion
            confirm = input(f"Are you sure you want to delete policy {policy_id}? (y/n): ").lower()
            
            if confirm == 'y':
                result = delete_cluster_policy(policy_id)
                
                if result is not None:
                    print(f"\nPolicy {policy_id} deleted successfully.")
                else:
                    print(f"\nFailed to delete policy {policy_id}.")
            else:
                print("\nDeletion cancelled.")
        
        elif choice == '6':
            # Get policy permissions
            policy_id = input("Enter policy ID: ")
            permissions = get_cluster_policy_permissions(policy_id)
            
            if permissions:
                print("\nPolicy permissions:")
                print(format_permissions(permissions))
            else:
                print(f"\nFailed to get permissions for policy {policy_id}.")
        
        elif choice == '7':
            # Set policy permissions
            policy_id = input("Enter policy ID: ")
            
            # Get current permissions first
            current_permissions = get_cluster_policy_permissions(policy_id)
            if current_permissions:
                print("\nCurrent permissions:")
                print(format_permissions(current_permissions))
            
            # Build new ACL
            acl = []
            acl_count = input("\nEnter number of access control entries: ")
            
            try:
                acl_count = int(acl_count)
                
                for i in range(acl_count):
                    print(f"\nEntry {i+1}:")
                    principal_type = input("Principal type (user/group/service_principal): ")
                    
                    if principal_type == 'user':
                        user_name = input("User name: ")
                        entry = {"user_name": user_name}
                    elif principal_type == 'group':
                        group_name = input("Group name: ")
                        entry = {"group_name": group_name}
                    elif principal_type == 'service_principal':
                        sp_name = input("Service principal name: ")
                        entry = {"service_principal_name": sp_name}
                    else:
                        print(f"Unsupported principal type: {principal_type}")
                        continue
                    
                    permission = input("Permission level (CAN_USE/CAN_MANAGE): ")
                    if not validate_permission_level(permission):
                        continue
                    
                    entry["permission_level"] = permission
                    acl.append(entry)
            except ValueError:
                print("Error: Number of entries must be an integer.")
                continue
            
            result = set_cluster_policy_permissions(policy_id, acl)
            
            if result:
                print("\nPermissions set successfully:")
                print(format_permissions(result))
            else:
                print(f"\nFailed to set permissions for policy {policy_id}.")
        
        elif choice == '8':
            # Update policy permissions
            policy_id = input("Enter policy ID: ")
            
            # Get current permissions first
            current_permissions = get_cluster_policy_permissions(policy_id)
            if current_permissions:
                print("\nCurrent permissions:")
                print(format_permissions(current_permissions))
            
            # Build ACL updates
            acl = []
            acl_count = input("\nEnter number of access control entries to update: ")
            
            try:
                acl_count = int(acl_count)
                
                for i in range(acl_count):
                    print(f"\nEntry {i+1}:")
                    principal_type = input("Principal type (user/group/service_principal): ")
                    
                    if principal_type == 'user':
                        user_name = input("User name: ")
                        entry = {"user_name": user_name}
                    elif principal_type == 'group':
                        group_name = input("Group name: ")
                        entry = {"group_name": group_name}
                    elif principal_type == 'service_principal':
                        sp_name = input("Service principal name: ")
                        entry = {"service_principal_name": sp_name}
                    else:
                        print(f"Unsupported principal type: {principal_type}")
                        continue
                    
                    permission = input("Permission level (CAN_USE/CAN_MANAGE): ")
                    if not validate_permission_level(permission):
                        continue
                    
                    entry["permission_level"] = permission
                    acl.append(entry)
            except ValueError:
                print("Error: Number of entries must be an integer.")
                continue
            
            result = update_cluster_policy_permissions(policy_id, acl)
            
            if result:
                print("\nPermissions updated successfully:")
                print(format_permissions(result))
            else:
                print(f"\nFailed to update permissions for policy {policy_id}.")
        
        elif choice == '9':
            print("\nExiting Cluster Policy Management CLI. Goodbye!")
            break
        
        else:
            print("\nInvalid choice. Please enter a number between 1 and 9.")

# Uncomment to run the interactive CLI
cluster_policy_cli()